In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import os
import glob
from pathlib import Path

# CSV 파일 읽기
csv_path = 'gyeonggi_welfare_services_PAJU.csv'
df = pd.read_csv(csv_path)

# SERVC_NM(서비스명)과 SERVC_RINK_ADDR(링크) 함께 추출
# 둘 다 값이 있는 행만 선택
df_filtered = df[['SERVC_NM', 'SERVC_RINK_ADDR']].dropna()

# 리스트로 변환
service_names = df_filtered['SERVC_NM'].tolist()
links = df_filtered['SERVC_RINK_ADDR'].tolist()

print(f"총 {len(links)}개의 링크를 찾았습니다.")
print("\n처음 3개 예시:")
for i in range(min(3, len(links))):
    print(f"  {i+1}. {service_names[i]}")
    print(f"     {links[i]}")


총 17개의 링크를 찾았습니다.

처음 3개 예시:
  1. 장애인가정 출산 지원
     https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId=WLF00001593&wlfareInfoReldBztpCd=02
  2. 독립유공자 유족 수당 및 유족 위문금
     https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId=WLF00004619&wlfareInfoReldBztpCd=02
  3. 파주시 보훈명예수당 지급
     https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId=WLF00004042&wlfareInfoReldBztpCd=02


In [ ]:
# 다운로드 폴더 설정
download_dir = os.path.join(os.getcwd(), 'downloads')
os.makedirs(download_dir, exist_ok=True)

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option('prefs', {
    'download.default_directory': download_dir,
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': True
})

# 필요시 주석 해제 (백그라운드 실행)
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')

print(f"다운로드 폴더: {download_dir}")

# 헬퍼 함수: 다운로드 완료 대기 및 최신 파일 찾기
def wait_for_download_and_get_file(download_dir, timeout=30):
    """다운로드가 완료될 때까지 기다리고 가장 최근 파일 반환"""
    seconds = 0
    while seconds < timeout:
        time.sleep(1)
        # .crdownload 또는 .tmp 파일이 없으면 다운로드 완료
        downloading = glob.glob(os.path.join(download_dir, '*.crdownload'))
        downloading += glob.glob(os.path.join(download_dir, '*.tmp'))
        
        if not downloading:
            break
        seconds += 1
    
    # 가장 최근에 다운로드된 파일 찾기
    files = glob.glob(os.path.join(download_dir, '*'))
    if files:
        latest_file = max(files, key=os.path.getctime)
        return latest_file
    return None

# 헬퍼 함수: 파일명을 안전하게 변환
def sanitize_filename(filename):
    """파일명에서 사용 불가능한 문자 제거"""
    invalid_chars = '<>:"/\\|?*'
    for char in invalid_chars:
        filename = filename.replace(char, '_')
    return filename


In [13]:
# 웹드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()

# 각 링크 방문하여 첨부파일 다운로드
success_count = 0
fail_count = 0
no_attachment_count = 0

for idx, (service_name, link) in enumerate(zip(service_names, links), 1):
    try:
        print(f"\n[{idx}/{len(links)}] 처리 중")
        print(f"  서비스명: {service_name}")
        print(f"  링크: {link}")
        
        # 다운로드 전 파일 목록 저장
        files_before = set(glob.glob(os.path.join(download_dir, '*')))
        
        # 페이지 로드
        driver.get(link)
        
        # 페이지 로드 대기
        time.sleep(3)
        
        # 첨부파일 찾기 - 여러 가능한 selector 시도
        attachment_found = False
        
        # WebDriverWait로 동적 콘텐츠 로딩 대기
        wait = WebDriverWait(driver, 10)
        
        selectors = [
            # 다운로드 텍스트를 포함하는 모든 요소
            "//*[contains(text(), '다운로드')]",
            "//a[contains(text(), '다운로드')]",
            "//button[contains(text(), '다운로드')]",
            "//div[contains(text(), '다운로드')]",
            "//div[contains(@class, 'cl-text') and contains(text(), '다운로드')]",
            
            # 첨부 관련
            "//*[contains(text(), '첨부')]",
            "//a[contains(text(), '첨부')]",
            "//a[contains(@class, 'attach')]",
            "//div[contains(@class, 'attach')]//a",
            "//span[contains(text(), '첨부')]//ancestor::a",
            
            # 첨부파일 관련
            "//*[contains(text(), '첨부파일')]",
            "//a[contains(text(), '첨부파일')]",
            
            # href에 download 포함
            "//a[contains(@href, 'download')]",
            "//a[contains(@href, 'Download')]",
            "//a[contains(@href, 'file')]",
            
            # 일반적인 다운로드 버튼/링크 클래스
            "//a[contains(@class, 'download')]",
            "//button[contains(@class, 'download')]",
            "//*[contains(@class, 'file-download')]",
            "//*[contains(@class, 'btn-download')]"
        ]
        
        for selector in selectors:
            try:
                attachment_elements = driver.find_elements(By.XPATH, selector)
                if attachment_elements:
                    print(f"  ✓ 첨부파일 발견 ({len(attachment_elements)}개) - selector: {selector[:40]}...")
                    
                    # 모든 첨부파일 클릭
                    for attach_idx, element in enumerate(attachment_elements, 1):
                        try:
                            # 요소가 화면에 보이고 클릭 가능할 때까지 대기
                            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
                            time.sleep(0.5)
                            
                            # 요소가 클릭 가능한지 확인
                            if element.is_displayed() and element.is_enabled():
                                # JavaScript로 클릭 시도 (더 안정적)
                                try:
                                    driver.execute_script("arguments[0].click();", element)
                                except:
                                    # JavaScript 클릭 실패시 일반 클릭
                                    element.click()
                                
                                print(f"    → 첨부파일 #{attach_idx} 다운로드 시작")
                                time.sleep(2)  # 다운로드 시작 대기
                                
                                attachment_found = True
                            else:
                                print(f"    ⚠ 첨부파일 #{attach_idx}가 클릭 불가능 상태")
                                
                        except Exception as e:
                            print(f"    ✗ 첨부파일 #{attach_idx} 클릭 실패: {str(e)[:50]}")
                    
                    if attachment_found:
                        break
            except Exception as e:
                continue
        
        if attachment_found:
            # 다운로드 완료 대기
            time.sleep(3)
            
            # 다운로드된 파일 찾기
            files_after = set(glob.glob(os.path.join(download_dir, '*')))
            new_files = files_after - files_before
            
            if new_files:
                # 가장 최근에 다운로드된 파일
                latest_file = max(new_files, key=os.path.getctime)
                
                # 원본 파일 확장자 가져오기
                file_ext = os.path.splitext(latest_file)[1]
                
                # 새 파일명 생성 (서비스명 + 확장자)
                safe_service_name = sanitize_filename(service_name)
                new_filename = f"{safe_service_name}{file_ext}"
                new_filepath = os.path.join(download_dir, new_filename)
                
                # 파일명이 중복되는 경우 번호 추가
                counter = 1
                while os.path.exists(new_filepath):
                    new_filename = f"{safe_service_name}_{counter}{file_ext}"
                    new_filepath = os.path.join(download_dir, new_filename)
                    counter += 1
                
                # 파일명 변경
                os.rename(latest_file, new_filepath)
                print(f"  ✓ 다운로드 완료: {new_filename}")
                success_count += 1
            else:
                print(f"  ⚠ 다운로드된 파일을 찾을 수 없습니다")
                no_attachment_count += 1
        else:
            no_attachment_count += 1
            print(f"  ⚠ 첨부파일을 찾을 수 없습니다")
            
    except Exception as e:
        fail_count += 1
        print(f"  ✗ 오류 발생: {str(e)[:100]}")
    
    # 너무 빠른 요청 방지
    time.sleep(1)

# 브라우저 종료
driver.quit()

print(f"\n{'='*60}")
print(f"처리 완료!")
print(f"  - 성공: {success_count}개")
print(f"  - 첨부파일 없음: {no_attachment_count}개")
print(f"  - 실패: {fail_count}개")
print(f"  - 다운로드 폴더: {download_dir}")
print(f"{'='*60}")



[1/17] 처리 중
  서비스명: 장애인가정 출산 지원
  링크: https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId=WLF00001593&wlfareInfoReldBztpCd=02
  ✓ 첨부파일 발견 (1개) - selector: //*[contains(text(), '다운로드')]...
    → 첨부파일 #1 다운로드 시작
  ✓ 다운로드 완료: 장애인가정 출산 지원.pdf

[2/17] 처리 중
  서비스명: 독립유공자 유족 수당 및 유족 위문금
  링크: https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId=WLF00004619&wlfareInfoReldBztpCd=02
  ✓ 첨부파일 발견 (1개) - selector: //*[contains(text(), '다운로드')]...
    → 첨부파일 #1 다운로드 시작
  ✓ 다운로드 완료: 독립유공자 유족 수당 및 유족 위문금.hwp

[3/17] 처리 중
  서비스명: 파주시 보훈명예수당 지급
  링크: https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId=WLF00004042&wlfareInfoReldBztpCd=02


KeyboardInterrupt: 